In [34]:
import re
import numpy as np
import pymorphy3

from sklearn.feature_extraction.text import TfidfVectorizer
from joblib import dump, load
import pickle

In [35]:
def save_file(name, obj, path):
    with open('{path}/{name}.pickle'.format(name=name, path=path), 'wb') as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [36]:
special_characters = ['</p>', '\xa0', '<br/>', '\n', '-&gt;']

In [37]:
stop_words = ['в', 'на', 'с', 'и', 'я', 'о', 'к', 'а', 'за', 'по',
              'об', 'у', 'бы', 'от', 'ли', 'ул', 'что', 'со', 'из',
              'для', 'про', 'г', 'до', 'то', 'быть', 'по', 'мочь',
              'январь','февраль','март','апрель','май','июнь','июль',
              'август','сентябрь','октябрь','ноябрь','декабрь','год',
              'стать','это','когда','даже', 'янв', 'фев', 'апр', 'авг',
              'сен', 'окт', 'ноя', 'дек', 'ые', 'ый', 'ым', 'ых', 'ая']


In [38]:
f = open("data/reviews_positive.txt", "r")
positive_reviews = f.read().split('<p>')
positive_reviews.pop(0)

f = open("data/reviews_neutral.txt", "r")
neutral_reviews = f.read().split('<p>')
neutral_reviews.pop(0)

f = open("data/reviews_negative.txt", "r")
negative_reviews = f.read().split('<p>')
negative_reviews.pop(0)

''

In [39]:
def cleansing(text):
    res = []
    for i in range(len(text)):
        res.append(text[i])
        for sp_ch in special_characters:
            res[i] = (text[i].replace(sp_ch,' '))
        res[i] = res[i].lower()
        res[i] = re.sub(r'[^\w\s]', ' ', res[i])
        res[i] = re.sub(r'[^\w\s]+|[\d]+', r' ',res[i]).strip()
        res[i] = re.sub('[a-zA-Z\s]+', ' ', res[i])
        res[i] = re.sub(" +", " ", res[i])
    return res

morph = pymorphy3.MorphAnalyzer()

def lemmatize(sentence):
    words = sentence.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)
    return res

def delete_stop_words(sentence):
    res = list()
    for word in sentence:
        if word not in stop_words:
            res.append(word)
    return res

def join(sentence):
    return ' '.join(sentence)


In [40]:
def processing_pipeline(reviews):
    cleansed_reviews = cleansing(reviews)
    lemmatized_reviews = list(map(lemmatize,cleansed_reviews))
    clean_reviews = list(map(delete_stop_words,lemmatized_reviews))
    final_reviews = list(map(join,clean_reviews))
    return final_reviews


In [41]:
positive_data = processing_pipeline(positive_reviews)
neutral_data = processing_pipeline(neutral_reviews)
negative_data = processing_pipeline(negative_reviews)

In [42]:
negative_reviews[0]

'Перешёл по ссылке друга оформил карту получил в ближайшем банке активно пользуюсь.в причине отказа оформления кэшбека пишут-вы не получили карту (просто недоумеваю).звонил на горячую линию ,обещали разобраться в течении 5 рабочих дней-прошло две недели.берут на измор,пока сам не отстанеш.</p>\n'

In [43]:
negative_data[0]

'перейти ссылка друг оформить карта получить близкий банк активно пользоваться причина отказ оформление кэшбек писать вы не получить карта просто недоумевать звонить горячий линия обещать разобраться течение рабочий день пройти два неделя брать измор пока сам не отстанеш'

In [44]:
vectorizer = TfidfVectorizer()

data = np.concatenate((positive_data,neutral_data))
data = np.concatenate((data,negative_data))
X = vectorizer.fit_transform(data)

dump(vectorizer, 'vectorizer.joblib')

['vectorizer.joblib']

In [45]:
save_file('data_positive', positive_data, 'prepared_data')
save_file('data_neutral', neutral_data, 'prepared_data')
save_file('data_negative', negative_data, 'prepared_data')